In [ ]:
from icecube import icetray, dataclasses, dataio, recclasses
from icecube.realtime_gfu.angular_errors import cramer_rao, paraboloid, PullCorrectorOnline2017
from icecube.realtime_gfu.muon_alerts import _replace_none, _clip_err50, _scale_err90
import numpy as np
import glob
import json

files = glob.glob("./search_data/ehe_events/candidate*.i3")
files.sort()
print(files)
dir_path = '/home/jovyan/alert_cat_v2/realtime_hese/resources/data/'

from realtime_hese.python.HESE_alerts_v2 import *
from realtime_ehe.python.EHE_alerts_v2 import *

In [ ]:
## event_dict = {}
for file in files:
    print(file)
    ofile = dataio.I3File(file)
    while ofile.more():
        frame = ofile.pop_physics()
        fm_hese = frame['FilterMask']['EHEAlertFilter_15']
        if fm_hese.condition_passed:
            header = frame['I3EventHeader']
            mjd = header.start_time.mod_julian_day_double
            print('mjd:',mjd)
            ehe_alert = json.loads(frame['AlertShortFollowupMsg'].value)
            #print(ehe_alert)
            #ehe_val = ehe_alert_eval(ehe_alert)
            #print(ehe_val)
            zen = round(frame["OnlineL2_SplineMPE"].dir.zenith, 5)
            #cos_zen = np.cos(zen)
            azi = round(frame["OnlineL2_SplineMPE"].dir.azimuth, 5)
            mjd = header.start_time.mod_julian_day_double
            truncatedEnergyName = 'OnlineL2_SplineMPE_TruncatedEnergy_AllDOMS_Muon'
            truncatedEnergy = round(frame[truncatedEnergyName].energy,5) if frame.Has(truncatedEnergyName) else None
            if truncatedEnergy is None:
                truncatedEnergyName = 'OnlineL2_SplineMPE_TruncatedEnergy_AllBINS_Muon'
                truncatedEnergy = round(frame[truncatedEnergyName].energy,5)
            print(truncatedEnergy)
            lognpe = math.log10(ehe_alert['ehe']['portia_npe'])
            cos_zen = math.cos(ehe_alert['reco']['imp_linefit']['zen'])
            ra, dec = astro.dir_to_equa(zen, azi, mjd)
            print(ra,dec,cos_zen,lognpe,ehe_alert['ehe']['portia_npe'])
            signalness = EHE_signalness(cos_zen,lognpe)
            exp_rate = float(EHE_far(np.log10(truncatedEnergy)))
            nu_energy = 10**EHE_energy_estimate(np.log10(truncatedEnergy))
            print(f'Signal:{signalness}, FAR:{exp_rate}, E_nu:{nu_energy}')
            run_id = header.run_id
            event_id = header.event_id
            event_sub_id = header.sub_event_id
            select = 'Gold' # default for ehe..
            cr_zen    = ehe_alert['reco']['splinempe']['cr_zen']
            cr_azi    = ehe_alert['reco']['splinempe']['cr_azi']
            pb_err1   = _replace_none(ehe_alert['reco']['splinempe']['pb_err1'], np.nan)
            pb_err2   = _replace_none(ehe_alert['reco']['splinempe']['pb_err2'], np.nan)
            bs_est    = _replace_none(ehe_alert['reco']['splinempe']['bs_est'],  np.nan)
            cr_est  = cramer_rao(cr_zen, cr_azi, zen)
            pb_est  = None if (pb_err1 is None) or (pb_err2 is None) else paraboloid(pb_err1, pb_err2)
            corrector = PullCorrectorOnline2017('TruncatedEnergy')
            err_50  = corrector(np.log10(truncatedEnergy), cr_est, pb_est, bs_est)[0]

            # clip 50% angular error
            err_50 = err_50.clip(_clip_err50)
            # rescale to 90% angular error
            err_90 = err_50 * _scale_err90
            print(err_50,err_90)
            print(run_id, event_id)
#            #print(header.start_time.mod_julian_day_double, header.run_id, header.event_id, ra, dec)
#            #print('HESE cqtot:',cqtot, ldir, cos_zen, truncatedEnergy, llh_ratio)
#                #print ('Alert:', pass_loose, pass_tight)
#                #print(header.start_time.mod_julian_day_double, header.run_id, header.event_id)
#                #print('HESE cqtot:',cqtot, ldir, cos_zen, truncatedEnergy, llh_ratio)
#                print('File:',file)
#                print(f"Run: {run_id:6} Event: {event_id:8} Subevent:{event_sub_id:3} MJD: {mjd:0.2f} ",\
#                      f"RA: {np.rad2deg(ra):0.2f} DEC: {np.rad2deg(dec):0.2f} ERR50: {np.rad2deg(err_50):0.2f} ERR90: {np.rad2deg(err_90):0.2f} ",\
#                      f"E_muon:{(truncatedEnergy/1000.0):0.2f}  E_nu:{(nu_energy/1000.0):0.2f} Sgns: {sig_ness:0.2f}",\
#                      f"CQTOT: {cqtot:7.2f} Selection: {select:8}  {overlaps:10}")
            key = str(run_id) + '_' + str(event_id)
            print(key)                  
            event_dict[key] = {
                'MJD' : mjd,
                'RA' : np.rad2deg(ra),                    
                'DEC' : np.rad2deg(dec),
                'ERR50' : np.rad2deg(err_50),
                'ERR90' : np.rad2deg(err_90),
                'E_NU' : nu_energy,
                'SGNS' : signalness,
                'FAR' : exp_rate,
                'SELECTION' : select
                }
print('*******')
for key in event_dict.keys():
            print(key, event_dict[key])
with open('ehe_events.txt', 'w') as outfile:
    json.dump(event_dict, outfile)
                
